In [93]:
import os
import spacy
import pandas as pd
import re

In [95]:
# turns the data in to a list + sorts it
dirs = sorted(os.listdir("data"))

In [154]:
# prints the subfolders in data 
dirs
dirs_test = [dirs[0]]
filenames_test = ["0102.a1.txt"]

In [166]:
# for every subfolder
for directory in dirs:

    # get the individual filenames
    filenames = sorted(os.listdir(os.path.join("data", directory)))

    print(len(filenames))

    columns = ["NOUN", "VERB", "ADJ", "ADV", "PERSON", "LOC", "ORG"]

    final_results = pd.DataFrame(columns=columns)

    # for each individual file
    for text_file in filenames:
        filepath = os.path.join("data", directory, text_file)
       
        with open(filepath, encoding='latin1') as file:
            text = file.read()
            # print(text)
            text_cleaned = re.sub(r'\<[^>]*\>', "", text)
        
            # print(text_cleaned)

        doc = nlp(text_cleaned)
    
        person = set()
        loc = set()
        org = set()
        
        for word in doc.ents:

            if word.label_ == "PERSON":
                person.add(str(word))

            elif word.label_ == "LOC":
                loc.add(str(word))
            
            elif word.label_ == "ORG":
                org.add(str(word))


       
        annotations = []
        for token in doc: 
            annotations.append([token.text, token.pos_])

        df = pd.DataFrame(annotations, columns=["text", "pos"])

        text_pos = df.groupby("pos").count()
        # print(text_pos)

        df2 = df[df['pos'].isin(["NOUN", "VERB", "ADJ", "ADV"])] 

        text_pos = df2.groupby("pos").count()
        
        total_words = df[-df['pos'].isin(["SPACE", "SYM", "PUNCT", "NUM"])] 

        total_words = len(total_words)/10000

        # print(text_pos)      
       
        noun = round(text_pos["text"]["NOUN"]/total_words)
        verb = round(text_pos["text"]["VERB"]/total_words)
        adj = round(text_pos["text"]["ADJ"]/total_words)
        adv = round(text_pos["text"]["ADV"]/total_words)
        person = len(person)
        loc = len(loc)
        org = len(org)


        results = [noun, verb, adj, adv, person, loc, org]


        df_results = pd.DataFrame([results], columns=columns)

        final_results = pd.concat([final_results, df_results])

    # print(final_results)

    final_results.to_csv(f"out/{directory}.results.csv", index=False)




         

303
344
292
185
114
80
57
35
29
21
12
15
3
7
